In [1]:
import numpy as np
from matplotlib import pyplot as plt

np.set_printoptions(suppress=True)

In [78]:
from numpy import sin, cos, arctan2 as atan2, log, pi
Np = 10
Nl = 8

X = np.zeros((3, Np))
L = np.zeros((5, Nl, Np))
L[:2] = np.random.randn(2, Nl, Np)
L[2] = 1e6
L[4] = 1e6

def pf_update(X, L, l_px, r):
    # Do a particle filter update: first, evaluate likelihood of every landmark for every particle
    # X.shape is [3, NumParticles] (x, y, theta)
    # L.shape is [5, NumLandmarks, NumParticles] (x, y, p11, p12, p22)
    p_x = X[0]
    p_y = X[1]
    theta = X[2]
    l_x = L[0]
    l_y = L[1]
    p11 = L[2]
    p12 = L[3]
    p22 = L[4]
    
    k0 = cos(theta)
    k1 = l_y - p_y
    k2 = k0*k1
    k3 = sin(theta)
    k4 = l_x - p_x
    k5 = k3*k4
    k6 = k0*k4 + k1*k3
    k7 = l_px - atan2(k2 - k5, k6)
    k8 = -k2 + k5
    k9 = k0*k6 + k3*k8
    k10 = k6**2 + k8**2
    k11 = k10**(-2)
    k12 = k0*k8 - k3*k6
    k13 = k11*k12*(k12*p11 + k9*p12) + k11*k9*(k12*p12 + k9*p22) + r
    k14 = 1/k10
    
    # also compute some handy quantities
    LL = -0.5*log(4*pi**2*k13) - k7**2/k13

    # get the maximum likelihood
    i = np.argmax(LL, axis=0)
    j = np.arange(Np)
    LL = LL[i, j]
    y_k = k7[i, j]
    S = k13[i, j]
    H1 = k12[i, j]*k14[i, j]
    H2 = k14[i, j]*k9[i, j]
    p11 = L[2, i, j]
    p12 = L[3, i, j]
    p22 = L[4, i, j]
    
    # we should resample based on LL at this step, *then* update the EKFs!
    # although actually, we end up doing exactly the same amount of work, so maybe not
    # *shrug* ok, let's update EKF and then throw away some particles
    
    k0 = 1/S
    k1 = H2*p12
    k2 = k0*(H1*p11 + k1)
    k3 = H1*p12
    k4 = H2*p22
    k5 = k0*(k3 + k4)
    k6 = H1*k2 - 1
    L[0, i, j] += k2*y_k
    L[1, i, j] += k5*y_k
    L[2, i, j] = -k1*k2 - k6*p11
    L[3, i, j] = -k2*k4 - k6*p12
    L[4, i, j] = -k3*k5 - p22*(H2*k5 - 1)
    
    return LL

pf_update(X, L, 0.1, 0.1)


array([-7.80318667, -8.06230683, -8.08928983, -7.73324357, -8.03371159,
       -8.14997983, -8.31886401, -7.84531503, -7.75299181, -7.98508002])